In [ ]:
import font

In [ ]:
font.font_use_pgf()

In [ ]:
from accumulation import *

In [ ]:
# Record the starting time for further filtering
start = time.time()

In [ ]:
ls = numexp.LogSeries("Accumulation", save=True, logs="redirect")
ls.new()

fs = numexp.FigureSeries(save=True, logs=ls, generator=".pdf")

rs = numexp.ResultSeries(logs=ls, generator="Database")
rs.bind()

ts = numexp.TableSeries(save=True, logs=ls)

In [ ]:
# Create a problem object
p = accumulation_random()

# Create a method object directly from initialization function
m = MNumpyAccumulate()

# Set the optimal solution, and here the statistical information of `numpy.sum`
# is recorded for further comparison
e = ESet()
e.evaluate(p, m)

In [ ]:
# Set up a testing evaluator
e = ETest(export=rs.write, logs=ls)

In [ ]:
m = MForAccumulate()
e.evaluate(p, m)

m = MSumAccumulate()
e.evaluate(p, m)

m = MNumpyAccumulate()
e.evaluate(p, m)

In [ ]:
ts.new(5, 4)

first_row = ["\\texttt{for}", "\\texttt{sum}", "\\texttt{numpy.sum}"]
for i in range(3):
    ts.update(0, i + 1, first_row[i])

first_col = ["Size", "Sum", "Time", "Relative time"]
for i in range(4):
    ts.update(i + 1, 0, first_col[i])

col_tag = ["for", "sum", "numpy"]
row_key = ["size", "sigma", "total", "ratio"]
row_fmt = ["{:.1e}", "{:.5e}", "{:.5f}", "{:.3f}"]

for i in range(3):
    for j in range(4):
        ts.update(j + 1, i + 1, rs.apply(
            lambda s: [
                e for e in s
                if col_tag[i] in e["tags"]
                if e["end"] > start
            ][0][row_key[j]],
            lambda s: row_fmt[j].format(s)
        ))

ts.emit()

fs.figure()
fs.subplot(1, 1, 1)
fs.ax.bar(numpy.arange(3),
    [rs.apply(
        lambda s: [
            e for e in s
            if col_tag[i] in e["tags"]
            if e["end"] > start
        ][0]["total"],
    ) for i in range(3)],
0.2)
fs.ax.set_xticks(numpy.arange(3))
fs.ax.set_xticklabels(first_row)
fs.emit()
fs.close()

ts.clean()

In [ ]:
rs.unbind()

ls.close()